In [1]:
import sys, os
import numpy as np
import pandas as pd
import statsmodels.stats.api as sms

sys.path.append("..")

ROOT_PATH = '/workspace/delta-interpolator/logs/lafan/'
LOGS_PATH = os.path.join(ROOT_PATH, 'transformer')


In [2]:
from glob import glob
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

metrics = ['L2Q@5', 'L2Q@15', 'L2Q@30', 
           'L2P@5', 'L2P@15', 'L2P@30', 
           'NPSS@5', 'NPSS@15', 'NPSS@30', 
          ]


def read_experiment_data(experiement_dir, experiment_pattern, metrics=None):
    experiemnt_folder_list = glob(os.path.join(experiement_dir, experiment_pattern))
    experiment_data_tst = {}
    index_max_tst = 0
    
    i=0    
    for experiement in experiemnt_folder_list:
        print(experiement)
        df_tst=[]
        df_val=[]
        for file in glob(os.path.join(experiement, 'events.out.tfevents.*')):
            ea = EventAccumulator(file)
            ea.Reload()
            tags = ea.Tags()['scalars']
            # print(tags)
            
            if not 'benchmark/'+metrics[0] in tags:
                continue 
                
            columns = []
            for m in metrics:
                df = pd.DataFrame(ea.Scalars('benchmark/'+m)).set_index('step').drop('wall_time', axis=1)
                df.columns = [m]
                columns.append(df)
            data = pd.concat(columns, axis=1)
            if len(data) > 0:
                df_tst.append(data)
         
        if len(df_tst) > 0:
            df_tst = pd.concat(df_tst).sort_index()
        
        if len(df_tst) > 0:
            experiment_data_tst[experiement+file] = df_tst
    
    return experiment_data_tst


def get_stats(df: pd.DataFrame, stats_steps = 100):
    df_observations = []
    for k, v in df.items():
        df_observations.append(v[-stats_steps:])

    df_observations = pd.concat(df_observations)
    df_stats = pd.DataFrame(df_observations.mean(), columns=['mean'])
    df_stats['ci_low'] = sms.DescrStatsW(df_observations).tconfint_mean()[0]
    df_stats['ci_high'] = sms.DescrStatsW(df_observations).tconfint_mean()[1]
    
    return df_stats

## Validate dataset and benchmark by running the zero-velocity model

In [3]:
!python run.py --config=src/configs/zerovel.yaml

/opt/conda/lib/python3.7/site-packages/hydra/core/utils.py:143: UserWarning: register_resolver() is deprecated.
See https://github.com/omry/omegaconf/issues/426 for migration instructions.

  OmegaConf.register_resolver(name, f)
/opt/conda/lib/python3.7/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/optimizers.py:38: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  UserWarning,

  | Name | Type                | Par

=================== LAFAN BENCHMARK ========================
L2Q@5 L2Q@15 L2Q@30 L2P@5 L2P@15 L2P@30 NPSS@5 NPSS@15 NPSS@30
0.562 1.0960 1.5060 1.512 3.6730 6.5580 0.0053 0.05210 0.23240


## Validate dataset and benchmark by running the SLERP interpolator model

In [4]:
!python run.py --config=src/configs/interpolator.yaml

/opt/conda/lib/python3.7/site-packages/hydra/core/utils.py:143: UserWarning: register_resolver() is deprecated.
See https://github.com/omry/omegaconf/issues/426 for migration instructions.

  OmegaConf.register_resolver(name, f)
/opt/conda/lib/python3.7/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/optimizers.py:38: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  UserWarning,

  | Name | Type                  | P

=================== LAFAN BENCHMARK ========================
L2Q@5 L2Q@15 L2Q@30 L2P@5 L2P@15 L2P@30 NPSS@5 NPSS@15 NPSS@30
0.220 0.6160 0.9730 0.373 1.2360 2.2890 0.0023 0.03900 0.20610


## The Key Result

In [5]:
experiment = 'Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_1
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_2
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_3
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_6
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.11377,0.31748,0.57233,0.12902,0.46702,0.99543,0.0014,0.02170,0.12172
ci_low,0.11359,0.31715,0.57162,0.12870,0.46624,0.99363,0.0014,0.02167,0.12125
ci_high,0.11396,0.31780,0.57303,0.12935,0.46781,0.99722,0.0014,0.02172,0.12219


## The ablation of the delta-mode

In [6]:
experiment = 'Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_5
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_6
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_1
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_0
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-inpu

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.11947,0.32535,0.57770,0.14050,0.4857,1.01350,0.00148,0.02213,0.12168
ci_low,0.11922,0.32486,0.57666,0.14015,0.4845,1.01159,0.00147,0.02209,0.12104
ci_high,0.11973,0.32584,0.57874,0.14085,0.4869,1.01541,0.00149,0.02217,0.12232


In [7]:
experiment = 'Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_4
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_6
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_0
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_5
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_none-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconst

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.15202,0.35420,0.61560,0.21835,0.55523,1.16545,0.0017,0.02376,0.12994
ci_low,0.15151,0.35353,0.61448,0.21658,0.55388,1.16006,0.0017,0.02369,0.12949
ci_high,0.15254,0.35487,0.61672,0.22012,0.55657,1.17084,0.0017,0.02382,0.13038


In [8]:
experiment = 'Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T


/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_0
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_3
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_2
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_4
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_6
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.11395,0.32195,0.58560,0.12960,0.48020,1.08618,0.0014,0.02214,0.12516
ci_low,0.11366,0.32139,0.58454,0.12935,0.47953,1.08240,0.0014,0.02210,0.12464
ci_high,0.11424,0.32251,0.58666,0.12985,0.48087,1.08995,0.0014,0.02218,0.12567


In [9]:
experiment = 'Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T


/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_3
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_1
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_2
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_5
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_last_pose-input_delta_mode_none-lr_0.00

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.12210,0.33483,0.59433,0.14430,0.51013,1.11897,0.0015,0.02274,0.12452
ci_low,0.12187,0.33410,0.59330,0.14391,0.50882,1.11595,0.0015,0.02268,0.12429
ci_high,0.12233,0.33555,0.59535,0.14469,0.51143,1.12200,0.0015,0.02281,0.12475


## The ablation of loss terms

In [10]:
experiment = 'Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, metrics=metrics)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_6
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_0
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_1
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_4
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode

,L2Q@5,L2Q@15,L2Q@30,L2P@5,L2P@15,L2P@30,NPSS@5,NPSS@15,NPSS@30
mean,0.12937,0.33517,0.59186,0.14766,0.49634,1.02571,0.0015,0.02282,0.12472
ci_low,0.12909,0.33464,0.59106,0.14723,0.49511,1.02316,0.0015,0.02277,0.12431
ci_high,0.12965,0.33570,0.59266,0.14808,0.49758,1.02827,0.0015,0.02287,0.12512


## The key frame metrics


In [13]:
metrics_key_frame = ['L2Q_KEY@5', 'L2Q_KEY@15', 'L2Q_KEY@30', 
                     'L2P_KEY@5', 'L2P_KEY@15', 'L2P_KEY@30', 
                     'NPSS_KEY@5', 'NPSS_KEY@15', 'NPSS_KEY@30', 
                    ]

In [14]:
experiment = 'Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), 
                          experiment_pattern=experiment, metrics=metrics_key_frame)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_1
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_3
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_5
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_0
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_1.0-dp_0.2-seed_2
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode

,L2Q_KEY@5,L2Q_KEY@15,L2Q_KEY@30,L2P_KEY@5,L2P_KEY@15,L2P_KEY@30,NPSS_KEY@5,NPSS_KEY@15,NPSS_KEY@30
mean,0.12997,0.14192,0.14812,0.06586,0.06822,0.07081,0.02205,0.02454,0.02687
ci_low,0.11751,0.12864,0.13461,0.06246,0.06466,0.06717,0.01977,0.02215,0.02448
ci_high,0.14243,0.15520,0.16163,0.06926,0.07179,0.07446,0.02432,0.02693,0.02925


In [15]:
experiment = 'Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_*'

df = read_experiment_data(experiement_dir=os.path.join(ROOT_PATH, LOGS_PATH), experiment_pattern=experiment, 
                          metrics=metrics_key_frame)
get_stats(df, stats_steps=5).round(5).T

/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_2
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_7
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_3
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_4
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode_interpolator-input_delta_mode_last_pose-lr_0.0002-bsz_64-enc_blocks_6-reconstruction_scale_0.0-dp_0.2-seed_5
/workspace/delta-interpolator/logs/lafan/transformer/Transformer-delta_mode

,L2Q_KEY@5,L2Q_KEY@15,L2Q_KEY@30,L2P_KEY@5,L2P_KEY@15,L2P_KEY@30,NPSS_KEY@5,NPSS_KEY@15,NPSS_KEY@30
mean,0.86344,0.77261,0.74194,2.64741,2.47799,2.37101,0.03800,0.04503,0.05253
ci_low,0.85033,0.76656,0.73911,2.61728,2.46422,2.36373,0.03792,0.04495,0.05242
ci_high,0.87655,0.77866,0.74478,2.67755,2.49176,2.37829,0.03807,0.04511,0.05263
